In [1]:
# !pip install --upgrade boto3==1.39.8
# !sudo apt install antiword 
import io
import uuid
import os
import random
import nltk
import boto3
import json
import pytesseract
import textract
from pdf2image import convert_from_bytes
from PyPDF2 import PdfReader
from docx import Document
from io import BytesIO
from nltk.tokenize import sent_tokenize
from transformers import AutoTokenizer, AutoModel
import extract_msg
import tempfile
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from llama_index.core.node_parser import SemanticSplitterNodeParser
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.schema import TextNode, NodeRelationship, RelatedNodeInfo
from sentence_transformers import SentenceTransformer

# embed_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
# embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-mpnet-base-v2")
embed_model = AutoModel.from_pretrained("intfloat/e5-small-v2")

S3_BUCKET = "ml-legal-restricted"
EXCEL_PATH = "full_contracts_with_files.xlsx"
VECTOR_BUCKET_NAME = "legal-docs-vector-store"

nltk.download('punkt')
nltk.download('punkt_tab')
s3 = boto3.client('s3')
s3v = boto3.client("s3vectors", region_name="us-east-1")

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
def download_excel(bucket, key, sheet_name="Active Legal Contracts", column="Contract Number"):
    print(f"📥 Downloading Excel: s3://{bucket}/{key}")
    obj = s3.get_object(Bucket=bucket, Key=key)
    excel_data = obj['Body'].read()

    df = pd.read_excel(io.BytesIO(excel_data), sheet_name=sheet_name, engine='openpyxl')
    if column not in df.columns:
        raise ValueError(f"Column '{column}' not found in sheet '{sheet_name}'")

    contract_numbers = df[column].dropna().astype(str).str.strip().tolist()
    return contract_numbers, df, excel_data


def list_s3_files_for_contract(bucket, contract_number, prefix_base="contract-docs/"):
    prefix = f"{prefix_base}{contract_number}/"
    files = []
    paginator = s3.get_paginator("list_objects_v2")

    for page in paginator.paginate(Bucket=bucket, Prefix=prefix):
        for obj in page.get("Contents", []):
            files.append(obj["Key"])
    return files


def add_modified_sheet_with_files(wb, original_df, file_map):
    ws = wb.create_sheet("Active Legal Contracts + Files")
    headers = original_df.columns.tolist()
    max_files = max((len(files) for files in file_map.values()), default=0)
    headers += [f"File {i+1}" for i in range(max_files)]
    ws.append(headers)

    for idx, row in original_df.iterrows():
        contract_number = str(row["Contract Number"]).strip()
        files = [os.path.basename(f) for f in file_map.get(contract_number, [])]
        base_row = row.tolist()
        ws.append(base_row + files)


def add_s3_paths_sheet(wb, file_map, bucket):
    ws = wb.create_sheet("S3 File Paths")
    max_files = max((len(files) for files in file_map.values()), default=0)

    headers = ["Contract Number"] + [f"S3 File {i+1}" for i in range(max_files)]
    ws.append(headers)

    for contract, keys in file_map.items():
        s3_paths = [f"s3://{bucket}/{key}" for key in keys]
        ws.append([contract] + s3_paths)

def save_missing_contracts_to_csv(missing_contracts, output_path="missing_contracts.csv"):
    if not missing_contracts:
        print("✅ No missing contracts to save.")
        return

    df = pd.DataFrame(missing_contracts, columns=["Contract Number"])
    df.to_csv(output_path, index=False)
    print(f"📄 Missing contracts CSV saved to: {output_path}")
    
def process_entire_bucket(bucket, excel_key, output_path="full_contracts_with_files.xlsx"):
    contract_numbers, original_df, excel_bytes = download_excel(bucket, excel_key)

    file_map = {}
    missing_contracts = []

    print(f"\n🔍 Scanning {len(contract_numbers)} contract numbers across S3...")
    for idx, number in enumerate(contract_numbers, 1):
        if idx % 1000 == 0 or idx == 1:
            print(f"🔢 [{idx}/{len(contract_numbers)}] Scanning: {number}")
        files = list_s3_files_for_contract(bucket, number)

        if not files and len(number) < 8 and number.isdigit():
            padded = number.zfill(8)
            print(f"   ➕ Retrying with padded contract number: {padded}")
            files = list_s3_files_for_contract(bucket, padded)
            
        if files:
            file_map[number] = files
        else:
            missing_contracts.append(number)

    print("\n🧾 Preparing final Excel workbook...")
    wb = load_workbook(io.BytesIO(excel_bytes))
    add_modified_sheet_with_files(wb, original_df, file_map)
    add_s3_paths_sheet(wb, file_map, bucket)
    wb.save(output_path)
    print(f"✅ Final Excel saved: {output_path}")

    print("\n=== Final Summary ===")
    print(f"📄 Total contracts processed: {len(contract_numbers)}")
    print(f"✅ Contracts with files: {len(file_map)}")
    print(f"❌ Contracts with NO files found: {len(missing_contracts)}")
    if missing_contracts:
        print(f"🔍 Sample missing contract numbers: {missing_contracts[:5]}{'...' if len(missing_contracts) > 5 else ''}")
        save_missing_contracts_to_csv(missing_contracts)

if __name__ == "__main__":
    bucket = "ml-legal-restricted"
    excel_key = "tabularData/Active Legal Contracts 7-10-2025 1-17-09 PM.xlsx"
    process_entire_bucket(bucket, excel_key, output_path="full_contracts_with_files.xlsx")


In [2]:
from transformers import AutoTokenizer, AutoModel
import torch

class E5Embedder:
    def __init__(self, model_name="intfloat/e5-small-v2", device=None):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)
        self.model.eval()

    def get_text_embedding_batch(self, texts):
        texts = [f"passage: {text}" for text in texts]
        inputs = self.tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to(self.device)
        with torch.no_grad():
            outputs = self.model(**inputs)
            embeddings = outputs.last_hidden_state[:, 0]
            embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)
        return embeddings.cpu().numpy().tolist()

    def get_text_embedding(self, text):
        return self.get_text_embedding_batch([text])[0]

embed_model = E5Embedder("intfloat/e5-small-v2")


In [5]:
# to recreate vector index store

INDEX_NAME = "token-chunking"
# INDEX_NAME = "overlap-chunking"
# INDEX_NAME = "semantic-split-chunking"
VECTOR_DIM = 384
DISTANCE_METRIC = "cosine"
NON_FILTERABLE_KEYS = ['text']

response = s3v.delete_index(
    vectorBucketName=VECTOR_BUCKET_NAME,
    indexName=INDEX_NAME,
)

response = s3v.create_index(
    vectorBucketName=VECTOR_BUCKET_NAME,
    indexName=INDEX_NAME,
    dataType="float32",
    dimension=VECTOR_DIM,
    distanceMetric=DISTANCE_METRIC,
    metadataConfiguration={
        "nonFilterableMetadataKeys": NON_FILTERABLE_KEYS
    }
)

print(f"Created index: {response}")


Created index: {'ResponseMetadata': {'RequestId': '8ade74ec-0c69-4318-8f2e-920d27820736', 'HostId': '', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Tue, 22 Jul 2025 19:13:47 GMT', 'content-type': 'application/json', 'content-length': '2', 'connection': 'keep-alive', 'x-amz-request-id': '8ade74ec-0c69-4318-8f2e-920d27820736', 'access-control-allow-origin': '*', 'vary': 'origin, access-control-request-method, access-control-request-headers', 'access-control-expose-headers': '*'}, 'RetryAttempts': 0}}


In [6]:
def chunk_text(text, s3_path, token_limit=500, tokenizer_name="intfloat/e5-small-v2"):
    file_name = os.path.basename(s3_path)
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

    sentences = sent_tokenize(text)
    chunks, current, count = [], [], 0

    for sent in sentences:
        toks = tokenizer.tokenize(sent)
        if count + len(toks) > token_limit and current:
            chunks.append(" ".join(current))
            current, count = [], 0
        current.append(sent)
        count += len(toks)

    if current:
        chunks.append(" ".join(current))

    return [{"key": str(uuid.uuid4()), "metadata": {"text": chunk, "s3_path": s3_path, "file_name": file_name}} for chunk in chunks]

def chunk_text_with_overlap(text, s3_path, token_limit=500, chunk_overlap=50, tokenizer_name="intfloat/e5-small-v2"):
    file_name = os.path.basename(s3_path)
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

    sentences, chunks, curr, curr_toks = sent_tokenize(text), [], [], 0

    for sent in sentences:
        stoks = len(tokenizer.tokenize(sent))
        if stoks > token_limit:
            words = sent.split()
            i = 0
            while i < len(words):
                segment = words[i:i+token_limit]
                chunks.append(" ".join(segment))
                i += token_limit - chunk_overlap
            continue

        if curr_toks + stoks <= token_limit:
            curr.append(sent); curr_toks += stoks
        else:
            chunks.append(" ".join(curr))
            # build overlap
            overlap, tot = [], 0
            for s in reversed(curr):
                l = len(tokenizer.tokenize(s))
                if tot + l > chunk_overlap:
                    break
                overlap.insert(0, s); tot += l
            curr = overlap + [sent]
            curr_toks = sum(len(tokenizer.tokenize(s)) for s in curr)

    if curr:
        chunks.append(" ".join(curr))
    
    return [{"key": str(uuid.uuid4()), "metadata": {"text": chunk, "s3_path": s3_path, "file_name": file_name}} for chunk in chunks]

def chunk_with_semantic_split(text, s3_path, 
                              buffer_size=1,
                              breakpoint_percentile_threshold=95):

    from llama_index.core import Document

    metadata = {"s3_path": s3_path, "file_name": os.path.basename(s3_path)}
    doc = Document(text=text, metadata=metadata)

    parser = SemanticSplitterNodeParser.from_defaults(
        embed_model=embed_model,
        buffer_size=buffer_size,
        breakpoint_percentile_threshold=breakpoint_percentile_threshold,
        include_metadata=False,
        include_prev_next_rel=False
    )

    nodes = parser.get_nodes_from_documents([doc])
    texts = [n.text for n in nodes]

    result = []
    for n in nodes:
        unique_id = str(uuid.uuid4())
        result.append({
            "key": unique_id,
            "metadata": {
                "text": n.text,
                "s3_path": s3_path,
                "file_name": os.path.basename(s3_path)
            },
        })

    return result

In [7]:
SUPPORTED_EXTENSIONS = {".pdf", ".docx", ".txt", ".msg", ".doc"}

def list_supported_files(bucket, prefix=""):
    paginator = s3.get_paginator("list_objects_v2")
    all_files, supported, unsupported = [], [], []

    for page in paginator.paginate(Bucket=bucket, Prefix=prefix):
        for obj in page.get("Contents", []):
            key = obj["Key"]
            all_files.append(key)
            ext = os.path.splitext(key)[1].lower()
            if ext in SUPPORTED_EXTENSIONS:
                supported.append(key)
            else:
                unsupported.append(key)

    print("\n=== File Summary ===")
    print(f"📁 Total files: {len(all_files)}")
    print(f"✅ Supported files: {len(supported)}")
    print(f"❌ Unsupported: {len(unsupported)} (Sample: {unsupported[:5]})\n")
    return supported

def download_s3_file(bucket, key):
    response = s3.get_object(Bucket=bucket, Key=key)
    return io.BytesIO(response["Body"].read())

def extract_from_doc(file_io):
    try:
        file_io.seek(0)
        with tempfile.NamedTemporaryFile(delete=False, suffix=".doc") as temp_file:
            temp_file.write(file_io.read())
            temp_path = temp_file.name

        text = textract.process(temp_path).decode("utf-8").strip()
        os.remove(temp_path)
        return text
    except Exception as e:
        print(f"❗ DOC (textract) error: {e}")
        return ""


def extract_with_ocr(file_io):
    try:
        file_io.seek(0)
        images = convert_from_bytes(file_io.read())
        return "\n".join(pytesseract.image_to_string(img) for img in images).strip()
    except Exception as e:
        print(f"❗ OCR failed: {e}")
        return ""

def extract_from_pdf(file_io):
    try:
        reader = PdfReader(file_io)
        return "\n".join([p.extract_text() for p in reader.pages if p.extract_text()]).strip()
    except Exception as e:
        print(f"❗ PDF read error: {e}")
        return ""


def extract_from_docx(file_io):
    from docx import Document
    from io import BytesIO

    try:
        if hasattr(file_io, "read"):
            file_io.seek(0)
            content = file_io.read()
            bio = BytesIO(content)
            
            doc = Document(bio)
        else:

            bio = BytesIO(file_io)
            doc = Document(bio)
            
        text = "\n".join(p.text for p in doc.paragraphs).strip()
        return text
    except Exception as e:
        print(f"❗ DOCX error: {e}")
        return ""


def extract_from_txt(file_io):
    try:
        return file_io.read().decode("utf-8").strip()
    except Exception as e:
        print(f"❗ TXT read error: {e}")
        return ""

def extract_from_msg(file_io):
    try:
        with open("temp.msg", "wb") as f:
            f.write(file_io.read())
        msg = extract_msg.Message("temp.msg")
        text = msg.body or ""
        os.remove("temp.msg")
        return text.strip()
    except Exception as e:
        print(f"❗ MSG read error: {e}")
        return ""

def extract_text(file_io, ext):
    ext = ext.lower()
    
    file_io.seek(0)
    sig = file_io.read(4)
    file_io.seek(0)
    
    extractors = {
        ".pdf": [
            extract_from_pdf,
            extract_with_ocr
        ],
        ".docx": [
            extract_from_docx,
            extract_with_ocr
        ],
        ".doc": [extract_from_doc],
        ".txt": [extract_from_txt,],
        ".msg": [extract_from_msg],
    }

    for extractor in extractors.get(ext, []):
        file_io.seek(0)
        text = extractor(file_io)
        if text:
            return text, extractor.__name__

    return "", "none"


def upload_chunks_to_s3_vector_index(chunks, embed_model, vector_bucket_name, chunking_strategy):

    texts = [chunk["metadata"]["text"] for chunk in chunks]
    keys = [chunk["key"] for chunk in chunks]

    embeddings = embed_model.get_text_embedding_batch(texts)

    vectors = []
    for i in range(len(chunks)):
        vector_metadata = chunks[i]["metadata"].copy()
        vectors.append({
            "key": keys[i],
            "data": {"float32": embeddings[i]},
            "metadata": vector_metadata
        })

    response = s3v.put_vectors(
        vectorBucketName=vector_bucket_name,
        indexName=chunking_strategy,
        vectors=vectors
    )

    return response


def process_documents(bucket, keys, chunking_strategy):
    stats = {
        "processed": 0, "failed": 0,
        "pdf": 0, "docx": 0, "txt": 0, "msg": 0, "doc": 0,
        "ocr": 0, "none": 0
    }

    all_chunks = []

    for idx, key in enumerate(keys, 1):
        print(f"\n📄 Processing {idx}/{len(keys)}: {key}")
        ext = os.path.splitext(key)[1].lower()
        s3_path = f"s3://{bucket}/{key}"

        try:
            file_io = download_s3_file(bucket, key)
            text, method = extract_text(file_io, ext)

            if not text:
                print("⚠️ No text extracted.")
                stats["failed"] += 1
                stats["none"] += 1
                continue
            
            if chunking_strategy == 'token-chunking':
                chunks = chunk_text(text, s3_path)
            elif chunking_strategy == 'overlap-chunking':
                chunks = chunk_text_with_overlap(text, s3_path)
            elif chunking_strategy == 'semantic-split-chunking':
                chunks = chunk_with_semantic_split(text, s3_path)

            print(f"✅ Extracted {len(chunks)} chunks.")
            # sample = chunks[0]['text'][:80].replace('\n', ' ')
            sample = chunks
            # print(f"📝 Sample: {sample}...")

            all_chunks.extend(chunks)
            stats["processed"] += 1
            stats[ext.replace(".", "")] += 1
            if "ocr" in method: stats["ocr"] += 1

        except Exception as e:
            print(f"❗ Error: {e}")
            stats["failed"] += 1
            stats["none"] += 1

    print("\n=== Summary ===")
    first_keys = ['processed', 'failed']
    first_line = []
    second_line = []

    for k, v in stats.items():
        key_formatted = k.capitalize().replace('_', ' ')
        pair = f"{key_formatted}: {v}"
        if k.lower() in first_keys:
            first_line.append(pair)
        else:
            second_line.append(pair)

    print("  ".join(first_line))
    print("  ".join(second_line))

    return stats, all_chunks

def read_s3_keys_from_excel(excel_path, sample_size, sheet_name="S3 File Paths"):
    print(f"📥 Reading S3 paths from Excel: {excel_path}, sheet: {sheet_name}")
    df = pd.read_excel(excel_path, sheet_name=sheet_name, engine="openpyxl")

    file_columns = [col for col in df.columns if col.startswith("S3 File")]

    if not file_columns:
        raise ValueError(f"No columns starting with 'S3 File' found in sheet '{sheet_name}'")

    all_paths = []
    for col in file_columns:
        for cell in df[col].dropna():
            path = str(cell).strip()
            if path:
                all_paths.append(path)

    print(f"🔍 Total paths found: {len(all_paths)}")

    valid_keys = []
    invalid_paths = []

    for path in all_paths:
        if path.startswith("s3://"):
            parts = path.replace("s3://", "").split("/", 1)
            if len(parts) == 2 and parts[1].strip():
                valid_keys.append(parts[1])
            else:
                invalid_paths.append(path)
        else:
            invalid_paths.append(path)

    print(f"📁 Parsed S3 keys: {len(valid_keys)}")

    if invalid_paths:
        print(f"⚠️ Skipped invalid paths: {len(invalid_paths)}")
        print("\n⚠️ Skipped Paths (sample):")
        for bad in invalid_paths[:10]:
            print(f" - {bad}")
        if len(invalid_paths) > 10:
            print(f" ...and {len(invalid_paths) - 10} more.")

    selected_keys = random.sample(valid_keys, min(sample_size, len(valid_keys)))
    print(f"🎯 Randomly selected {len(selected_keys)} files.")
    return selected_keys

if __name__ == "__main__":

    # file_keys = read_s3_keys_from_excel(EXCEL_PATH, sample_size=5)
    file_keys = ['contract-docs/66179/Dynamics_-_66179_signed.pdf', 'contract-docs/77767/Dynamics_-_77767_signed.pdf', 'contract-docs/65062/33908_SIGNED_ESI CO# 3417pdf.pdf', 'contract-docs/57964/VNSNY-Cotiviti Amend No 3 to MSLA Client Release 3.0 (SBC 02.04.20).docx', 'contract-docs/55622/Highmark - SOW# 1 Prospective Claims (March 2016) [55622].pdf', 'contract-docs/80537/VRC EL - BEV Equity Valuation as of 12-31-22 - LEGAL APPROVED - 22Feb2023_Fully executed.pdf', 'contract-docs/64936/24380_WellCare CO 3266 - CO 12 to PQ 13 - Aetna LOBs and Plan Codes (8249).pdf', 'contract-docs/80577/Vena Solutions_NDA_03292017_vendor signed.pdf', 'contract-docs/67566/Ochsner - Cotiviti - NDA - MV - 02222022 - Dynamics_67566.pdf', 'contract-docs/54094/Isos-Cotiviti- PAD Migration SOW - ATS2018720 - Ready for Signature.pdf', 'contract-docs/59747/Cotiviti - NCQA License and Certification Agreement Amendment_8.17.20.pdf', 'contract-docs/64408/59062_SIGNED_Anthem CE - Ready Set Renew IVR SMS & Email Progam - SOW 880 Signature Ready 101921.pdf', 'contract-docs/00010217/Willis-Mosiac-Towers Watson DUA 01-22-2014.docx', 'contract-docs/53985/Partners Healthcare MSA Amendment 4 (DxCG) jsw 12.18.18 FINAL_signed.pdf', 'contract-docs/78927/Dynamics_78927_signed.pdf', 'contract-docs/59164/RStudio Renewal - Q-16631-20200402-1828 - Preeti Vaidya.pdf', 'contract-docs/71355/Dynamics_-_71355_signed.pdf', 'contract-docs/64596/28695_SIGNED_Boston Medical Center HealthNet Plan CO1 to SOW3 (8778) .pdf', 'contract-docs/61941/Dynamics_61941_Please_review_and_sign_your_do_signed.pdf', 'contract-docs/00037577/Re  Requisition for Invite Networks  quote # 324MJ-Q   SoJo switches.msg', 'contract-docs/62520/Approval Needed_ Temp Contract Extension (Compliance - Stinson, Leticia) - FP&A - 07.29.2021.pdf', 'contract-docs/00006573/Towers Watson-MI-NDA-Executed.pdf', 'contract-docs/75036/Dynamics_75036_signed.pdf', 'contract-docs/71954/InSync SOW Rochelle Gilpin .docx', 'contract-docs/74972/Brex - Cotiviti - NDA - MS - 07092024 - Dynamics_74972.pdf']

    for chunking_strategy in ["token-chunking"]: #, "overlap-chunking", "semantic-split-chunking"]:
        final_stats, chunks = process_documents(S3_BUCKET, file_keys, chunking_strategy)
        if chunks:
            response = upload_chunks_to_s3_vector_index(chunks, embed_model, VECTOR_BUCKET_NAME, chunking_strategy)
            if response:
                print(f"\n ✅ Successfully uploaded to {chunking_strategy} S3 vector Index")




📄 Processing 1/25: contract-docs/66179/Dynamics_-_66179_signed.pdf


Token indices sequence length is longer than the specified maximum sequence length for this model (658 > 512). Running this sequence through the model will result in indexing errors


✅ Extracted 17 chunks.

📄 Processing 2/25: contract-docs/77767/Dynamics_-_77767_signed.pdf


Token indices sequence length is longer than the specified maximum sequence length for this model (722 > 512). Running this sequence through the model will result in indexing errors


✅ Extracted 7 chunks.

📄 Processing 3/25: contract-docs/65062/33908_SIGNED_ESI CO# 3417pdf.pdf
✅ Extracted 2 chunks.

📄 Processing 4/25: contract-docs/57964/VNSNY-Cotiviti Amend No 3 to MSLA Client Release 3.0 (SBC 02.04.20).docx
✅ Extracted 4 chunks.

📄 Processing 5/25: contract-docs/55622/Highmark - SOW# 1 Prospective Claims (March 2016) [55622].pdf


...

✅ Extracted 11 chunks.

📄 Processing 20/25: contract-docs/00037577/Re  Requisition for Invite Networks  quote # 324MJ-Q   SoJo switches.msg
✅ Extracted 9 chunks.

📄 Processing 21/25: contract-docs/62520/Approval Needed_ Temp Contract Extension (Compliance - Stinson, Leticia) - FP&A - 07.29.2021.pdf
✅ Extracted 3 chunks.

📄 Processing 22/25: contract-docs/00006573/Towers Watson-MI-NDA-Executed.pdf
✅ Extracted 5 chunks.

📄 Processing 23/25: contract-docs/75036/Dynamics_75036_signed.pdf


Token indices sequence length is longer than the specified maximum sequence length for this model (969 > 512). Running this sequence through the model will result in indexing errors


✅ Extracted 6 chunks.

📄 Processing 24/25: contract-docs/71954/InSync SOW Rochelle Gilpin .docx
✅ Extracted 1 chunks.

📄 Processing 25/25: contract-docs/74972/Brex - Cotiviti - NDA - MS - 07092024 - Dynamics_74972.pdf


Token indices sequence length is longer than the specified maximum sequence length for this model (755 > 512). Running this sequence through the model will result in indexing errors


✅ Extracted 12 chunks.

=== Summary ===
Processed: 25  Failed: 0
Pdf: 21  Docx: 3  Txt: 0  Msg: 1  Doc: 0  Ocr: 1  None: 0

 ✅ Successfully uploaded to token-chunking S3 vector Index


In [8]:
#check number of chunks uploaded

INDEX_NAME = "token-chunking"
# INDEX_NAME = "overlap-chunking"
# INDEX_NAME = "semantic-split-chunking"
response = s3v.list_vectors(
    vectorBucketName=VECTOR_BUCKET_NAME,
    indexName=INDEX_NAME,
    returnData=True,
    returnMetadata=True
)

vectors = response.get("vectors", [])
print(len(vectors))

# for vector in vectors:
#     print(f"Key: {vector['key']}")
#     print(f"Metadata: {vector.get('metadata',{})}")
#     print(f"Embedding (first 5 dims): {vector['data']['float32'][:5]}")
#     print("--------------")

# next_token = response.get("nextToken")
# if next_token:
#     pass


197


In [9]:
def query_s3_vector_store(query_text, INDEX_NAME, top_k = 5):
    print(f"\n--- Processing Query: '{query_text}' ---")

    query_embedding = embed_model.get_text_embedding(query_text)

    try:
        response = s3v.query_vectors(
            vectorBucketName=VECTOR_BUCKET_NAME,
            indexName=INDEX_NAME,
            topK=top_k,
            queryVector={
                'float32': query_embedding
            },
            returnMetadata=True,
            returnDistance=True
        )
        return response
    except Exception as e:
        print(f"Error querying S3 Vector Store: {e}")
        return None

user_questions = [
    "What obligations does Cotiviti have under Schedule C for Prepay FWAV Services?",
    "Under Schedule C, what services is Cotiviti required to provide?",
    "In the Prepay FWAV Services section, what are Cotiviti's main deliverables?",
]


INDEX_NAME = "token-chunking"
# INDEX_NAME = "overlap-chunking"
# INDEX_NAME = "semantic-split-chunking"

for question in user_questions:
    query_results = query_s3_vector_store(question, INDEX_NAME, top_k=5)

    if query_results and 'vectors' in query_results:
        print(f"Retrieved {len(query_results['vectors'])}")
        for i, chunk_data in enumerate(query_results['vectors']):
            # print(f"  Chunk {i+1}:")
            # print(f"    Text: {chunk_data['metadata'].get('text', 'No text found in metadata')[:50]}...")
            print(f"    S3_Path {i+1}: {chunk_data['metadata'].get('s3_path', 'No S3 path found')}")
            # print(f"    Distance: {chunk_data.get('distance', 'N/A')}")
    else:
        print(f"No results or an error occurred for query: '{question}'")



--- Processing Query: 'What obligations does Cotiviti have under Schedule C for Prepay FWAV Services?' ---
Retrieved 5
    S3_Path 1: s3://ml-legal-restricted/contract-docs/71355/Dynamics_-_71355_signed.pdf
    S3_Path 2: s3://ml-legal-restricted/contract-docs/66179/Dynamics_-_66179_signed.pdf
    S3_Path 3: s3://ml-legal-restricted/contract-docs/66179/Dynamics_-_66179_signed.pdf
    S3_Path 4: s3://ml-legal-restricted/contract-docs/66179/Dynamics_-_66179_signed.pdf
    S3_Path 5: s3://ml-legal-restricted/contract-docs/66179/Dynamics_-_66179_signed.pdf

--- Processing Query: 'Under Schedule C, what services is Cotiviti required to provide?' ---
Retrieved 5
    S3_Path 1: s3://ml-legal-restricted/contract-docs/66179/Dynamics_-_66179_signed.pdf
    S3_Path 2: s3://ml-legal-restricted/contract-docs/66179/Dynamics_-_66179_signed.pdf
    S3_Path 3: s3://ml-legal-restricted/contract-docs/71355/Dynamics_-_71355_signed.pdf
    S3_Path 4: s3://ml-legal-restricted/contract-docs/66179/Dynamics_-